#  Chunker de Textos - Experimento

* Este componente faz janelamento de textos longos, dividindo um texto em partes menores, cada parte de texto criada (chunk), possui uma quantidade de elementos (palavras ou sentenças) pré-definidas pelo usuário; como também a quantidade de elementos que se sobrepõem entre cada chunk consecutivo. 
* Exemplo:
        
    - Texto: "Hoje o dia amanheceu ensolarado, vou fazer uma caminhada e ouvir os sons da natureza."

    - Hiperparâmetros: {
        "chunkenizer": "word",
        "chunk_size": 5,
        "chunk_overlap": 2
    }

    - Saída: [
        "Hoje o dia amanheceu ensolarado,",
        "amanheceu ensolarado, vou fazer uma",
        "fazer uma caminhada e ouvir",
        "e ouvir os sons da",
        "sons da natureza."
    ]


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [23]:
dataset = "/tmp/data/texts-32.csv" #@param {type:"string"}
text_column_name = "text" #@param {type:"string", label:"Coluna para aplicar o Chunker", description:"Esta coluna será utilizada para criar os chunks de textos. Deve ser uma coluna existente do dataset."}
output_column_name = "text_chunk" #@param {type:"string", label:"Coluna para salvar os chunks gerados", description:"Esta coluna será utilizada para salvar os chunks de textos. Deve ser uma coluna inexistente do dataset."}
chunkenizer = "word" #@param ["word","sentence"] {type:"string",multiple:false,label:"Chunkenizer",description:"Tipo de chunkenizer para aplicar nos dados. Word separa textos em palavras, Sentence separa em sentenças."}
chunk_size = 96 #@param {type:"number",label:"Chunk Size",description:"Tamanho das janelas criadas pelo Chunker."}
chunk_overlap = 64 #@param {type:"number",label:"Chunk Overlap",description:"Tamanho da sobreposição das janelas criadas pelo Chunker."}
replicate_data = "sim" #@param ["sim","não"] {type:"string",multiple:false,label:"Replicar Dados",description:"Optando por replicar os dados, para cada chunk criado a partir de um campo de texto de uma linha da tabela, todos os dados dessa linha serão replicados de acordo com a quantidade de chunks produzidos."}

## Leitura do conjunto de dados

O exemplo abaixo faz a leitura de dados tabulares (ex: .csv).<br>
Modifique o código de acordo com o tipo de dado que desejar ler.

In [24]:
import pandas as pd

df = pd.read_csv(dataset)
df

,index,text,expected_text
0,1,A bola é um objeto utilizado para lazer de uma...,A ball is an object used for a person's leisur...
1,2,Golfe é um esporte no qual os jogadores usam d...,Golf is a sport in which players use different...
2,3,"Jogo por tacadas[1] (stroke play[2]), também c...","Stroke play[1] (stroke play[2]), also known as..."


## Conteúdo da tarefa

In [25]:
# Assert parameters

assert text_column_name in df.columns, f"Coluna '{text_column_name}' deve ser uma coluna do dataset."
assert output_column_name not in df.columns, f"Coluna '{output_column_name}' não pode estar contida nas colunas do dataset."
assert chunkenizer in ['word', 'sentence'], f"Chunkenizer '{chunkenizer}' deve ser ou 'word' ou 'sentence'."
assert chunk_size > chunk_overlap, f"'chunk_size' ({chunk_size}) deve ser um número maior que 'chunk_overlap' ({chunk_overlap})."
assert replicate_data in ['sim', 'não'], f"Replicar Dados deve ser ou 'sim' ou 'não'."

In [26]:
# Model parameters
model_parameters = {
    'chunkenizer':chunkenizer,
    'chunk_size':chunk_size,
    'chunk_overlap':chunk_overlap,
}

# Inference parameters
inference_parameters = {
    'text_column_name':text_column_name,
    'output_column_name':output_column_name,
    'columns':list(df.columns),
    'replicate_data': replicate_data
}

In [27]:
from chunker import Chunker

# Initilizate Chunker
chunker = Chunker(**model_parameters)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# Get chunks
chunks = chunker(df[text_column_name])

In [29]:
from utils import generate_dataframe

# Build output dataset
if inference_parameters['replicate_data'] == 'sim':
    # Replicate data
    new_df = generate_dataframe(df, chunks, inference_parameters)
    
else:
    new_df = df.copy()
    new_df[inference_parameters['output_column_name']] = chunks
    
new_df.head()

,index,text,expected_text,text_chunk
0,1,A bola é um objeto utilizado para lazer de uma...,A ball is an object used for a person's leisur...,[A bola é um objeto utilizado para lazer de um...
1,2,Golfe é um esporte no qual os jogadores usam d...,Golf is a sport in which players use different...,[Golfe é um esporte no qual os jogadores usam ...
2,3,"Jogo por tacadas[1] (stroke play[2]), também c...","Stroke play[1] (stroke play[2]), also known as...","[Jogo por tacadas[1] (stroke play[2]), também ..."


In [30]:
# Save data
new_df.to_csv(dataset, index=False)

## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [31]:
from joblib import dump

artifacts = {
    "model_parameters": model_parameters,
    "inference_parameters": inference_parameters
}

dump(artifacts, "/tmp/data/chunker.joblib")

['/tmp/data/chunker.joblib']